<h2> Analyzing Used Car Listings on eBay Kleinanzeigen </h2>

We will be working on a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. The version of the dataset we are working with is a sample of 50,000 data points that was prepared by Dataquest including simulating a less-cleaned version of the data.

The data dictionary provided with data is as follows:

* dateCrawled - When this ad was first crawled. All field-values are taken from this date.

* name - Name of the car.

* seller - Whether the seller is private or a dealer.

* offerType - The type of listing

* price - The price on the ad to sell the car.

* abtest - Whether the listing is included in an A/B test.

* vehicleType - The vehicle Type.

* yearOfRegistration - The year in which which year the car was first registered.

* gearbox - The transmission type.

* powerPS - The power of the car in PS.

* model - The car model name.

* kilometer - How many kilometers the car has driven.

* monthOfRegistration - The month in which which year the car was first registered.

* fuelType - What type of fuel the car uses.

* brand - The brand of the car.

* notRepairedDamage - If the car has a damage which is not yet repaired.

* dateCreated - The date on which the eBay listing was created.

* nrOfPictures - The number of pictures in the ad.

* postalCode - The postal code for the location of the vehicle.

* lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd

autos = pd.read_csv("autos.csv", encoding="Latin-1")

autos.head()
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

We see that  column names use camelcase instead of Python's preferred snakecase, let's change it to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [2]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Now let's take a deep look at our data in order to determine what needs to be done to clean it up. Some of the things we'll look for:

* Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.

* numeric data stored as text which can be cleaned and converted to do analysis.

In [3]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:45:59
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


Initial observations:
    
* There are a number of text columns where all (or nearly all) of the values are the same:
  - seller
  - offer_type
* The num_photos column looks odd, we'll need to investigate this further.
<!-- * price and odometer columns are numeric values stored as text 
  * How do we know this? no stats as to their mean, max, etc. There should be as there are numeric columns.  -->

In [4]:
print(autos["seller"].value_counts())
print("\n")
print(autos["offer_type"].value_counts())
print("\n")
print(autos["num_photos"].value_counts())

#It looks like the num_photos column has 0 for every column. 
#We'll drop this column, plus the other two we noted as mostly one value.
autos.drop(["offer_type", "seller", "num_photos"], axis=1)

privat        371525
gewerblich         3
Name: seller, dtype: int64


Angebot    371516
Gesuch         12
Name: offer_type, dtype: int64


0    371528
Name: num_photos, dtype: int64


,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,40764,2016-03-24 12:45:21


Let's continue exploring specially Price and Odometer.

In [5]:
autos["odometer_km"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: odometer_km, dtype: int64

In [6]:
print(autos["price"].describe())
print("\n ---------------------")
print(autos["price"].unique().shape)
print("\n ---------------------")
print(autos["price"].value_counts().head(20))

count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64

 ---------------------
(5597,)

 ---------------------
0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
999      3364
750      3203
650      3150
4500     3053
850      2946
2200     2936
700      2936
1800     2886
900      2874
950      2793
Name: price, dtype: int64


In [7]:
autos["price"].value_counts().sort_index(ascending=False).head(15)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
Name: price, dtype: int64

The prices in this column seem rounded, however given there are 2357 unique values in the column, that may just be people's tendency to round prices on the site.

There are 1,421 cars listed with $0 price - given that this is only 2% of the of the cars, we might consider removing these rows. The maximum price is more than 2 billion dollars, which seems a lot.

In [8]:
autos["price"].value_counts().sort_index().head(15)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
Name: price, dtype: int64

There are a number of listings with prices below 30 dollars including 1500 at 0.


Given that eBay is an auction site, there could legitimately be items where the opening bid is $1. We will keep the $1 items, but remove anything above 350,000 since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [9]:
autos = autos[autos["price"].between(1,350000)]
autos["price"].describe()

count    360635.000000
mean       5898.671956
std        8866.359669
min           1.000000
25%        1250.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

<h1> Exploring the date columns </h1>

There are a number of columns with date information:

* date_crawled
* registration_month
* registration_year
* ad_created
* last_seen


In [10]:
autos[["date_crawled", "registration_month", "registration_year", "ad_created", "last_seen"]][:5]

,date_crawled,registration_month,registration_year,ad_created,last_seen
0,2016-03-24 11:52:17,0,1993,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,5,2011,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,8,2004,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,6,2001,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,7,2008,2016-03-31 00:00:00,2016-04-06 10:17:21


In [11]:
print(autos["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index().head(20))
print("\n")
print(autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index().head(15))
print("\n")
print(autos["last_seen"].str[:10].value_counts(normalize=True, dropna=False).sort_index().head(15))


2016-03-05    0.025547
2016-03-06    0.014483
2016-03-07    0.035657
2016-03-08    0.033469
2016-03-09    0.034115
2016-03-10    0.032645
2016-03-11    0.032773
2016-03-12    0.036242
2016-03-13    0.015783
2016-03-14    0.036330
2016-03-15    0.033424
2016-03-16    0.030205
2016-03-17    0.031647
2016-03-18    0.013119
2016-03-19    0.035271
2016-03-20    0.036400
2016-03-21    0.035682
2016-03-22    0.032493
2016-03-23    0.032002
2016-03-24    0.029914
Name: date_crawled, dtype: float64


2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
2015-08-10    0.000003
2015-09-04    0.000006
2015-09-09    0.000003
2015-10-14    0.000003
2015-11-02    0.000003
2015-11-08    0.000003
2015-11-10    0.000003
2015-11-12    0.000003
2015-11-13    0.000003
2015-11-17    0.000003
Name: ad_created, dtype: float64


2016-03-05    0.001264
2016-03-06    0.004098
2016-03-07    0.005202
2016-03-08    0.007939
2016-03-09    0.009824
2016-03-

Looks like the site was crawled and last seen on a specific month - march of 2016. The add was created at different times.

** Dealing with Incorrect Registration Year Data**

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [12]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038751091824143526

Given that this is less than 4% of our data, we will remove these rows.

In [13]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.066699
1999    0.063552
2005    0.062669
2006    0.057708
2001    0.056955
2003    0.056557
2004    0.056173
2002    0.054290
2007    0.050499
1998    0.049691
Name: registration_year, dtype: float64

It appears that most of the vehicles were first registered in the past 22 years.

<h1> Exploring Price By Brand </h1>


In [14]:
autos["brand"].value_counts(normalize=True).sort_values(ascending=False)

volkswagen        0.211700
bmw               0.109871
opel              0.106410
mercedes_benz     0.096841
audi              0.089543
ford              0.068918
renault           0.047516
peugeot           0.030153
fiat              0.025691
seat              0.018661
skoda             0.015687
mazda             0.015384
smart             0.014331
citroen           0.013950
nissan            0.013598
toyota            0.012932
hyundai           0.009972
sonstige_autos    0.009493
mini              0.009384
volvo             0.009147
mitsubishi        0.008236
honda             0.007532
kia               0.006915
suzuki            0.006364
alfa_romeo        0.006309
porsche           0.006211
chevrolet         0.005022
chrysler          0.003863
dacia             0.002495
jeep              0.002192
land_rover        0.002166
daihatsu          0.002161
subaru            0.002117
jaguar            0.001734
saab              0.001465
daewoo            0.001457
trabant           0.001408
l

German manufacturers represent four out of the top five brands with Volswagen the most popular brand.

Let's now determine the avergae price for each brand

In [15]:
autos_brands = autos["brand"].value_counts(normalize=True).sort_values(ascending=False).index
brand_avg_price = {}

for i in autos_brands:
    brand = autos[autos["brand"] == i]
    price = brand["price"].mean()
    brand_avg_price[i] = int(price)



brand_avg_price = {k: i for k, i in sorted(brand_avg_price.items(), key=lambda x: x[1], reverse=True)}
brand_avg_price


{'porsche': 42258,
 'land_rover': 17070,
 'sonstige_autos': 14288,
 'jaguar': 13765,
 'jeep': 11213,
 'mini': 10080,
 'audi': 9086,
 'mercedes_benz': 8551,
 'bmw': 8449,
 'chevrolet': 7117,
 'skoda': 6530,
 'dacia': 5922,
 'kia': 5855,
 'hyundai': 5567,
 'volkswagen': 5400,
 'toyota': 5339,
 'volvo': 5238,
 'nissan': 4708,
 'seat': 4541,
 'subaru': 4386,
 'alfa_romeo': 4291,
 'chrysler': 4121,
 'mazda': 4076,
 'suzuki': 4044,
 'honda': 4005,
 'saab': 3955,
 'citroen': 3734,
 'ford': 3696,
 'smart': 3632,
 'mitsubishi': 3407,
 'lancia': 3289,
 'peugeot': 3267,
 'lada': 3191,
 'opel': 2971,
 'fiat': 2892,
 'renault': 2437,
 'trabant': 1900,
 'daihatsu': 1775,
 'rover': 1600,
 'daewoo': 1027}

We see that Porsche has the highest avergae price while daweoo has the lowest.

<h1> Exploring Mileage </h1>

Let's explore Mileage by brand

In [16]:
mileage = autos["odometer_km"].value_counts(normalize=True).sort_values(ascending=False).index

mileage_by_brand = {}

for i in autos_brands:
    brand = autos[autos["brand"] == i]
    mileage = brand["odometer_km"].mean()
    mileage_by_brand[i] = int(mileage)
    
mileage_by_brand = {x: i for x, i in sorted(mileage_by_brand.items(), key=lambda x: x[1], reverse=True)}
mileage_by_brand
    

{'saab': 141673,
 'volvo': 138131,
 'rover': 135204,
 'chrysler': 134215,
 'bmw': 132800,
 'mercedes_benz': 130572,
 'audi': 129443,
 'alfa_romeo': 129108,
 'opel': 128722,
 'volkswagen': 128386,
 'renault': 127885,
 'mitsubishi': 126959,
 'subaru': 126062,
 'mazda': 125754,
 'honda': 125748,
 'peugeot': 124500,
 'lancia': 124068,
 'daewoo': 123970,
 'ford': 123662,
 'jaguar': 122770,
 'seat': 120608,
 'jeep': 120453,
 'citroen': 120214,
 'nissan': 119446,
 'daihatsu': 119125,
 'land_rover': 119114,
 'toyota': 117271,
 'fiat': 116619,
 'skoda': 113593,
 'kia': 109144,
 'suzuki': 106729,
 'hyundai': 104080,
 'chevrolet': 100146,
 'smart': 99505,
 'porsche': 98980,
 'mini': 93273,
 'sonstige_autos': 87468,
 'dacia': 86080,
 'lada': 74565,
 'trabant': 55471}